# Parse GeoJSON and construct links

This code replicates the functionality of the web map found at https://lidar.digitalforestry.org

User must install shapely module first. (`pip install shapely`)

In [2]:
import json
from shapely.geometry import shape

In [9]:
import json
from shapely.geometry import shape

def parse_geojson(geojson_path):
    with open(geojson_path, 'r') as file:
        data = json.load(file)
    return data

def calculate_center(geometry):
    """Calculate the centroid of a polygon."""
    shapely_polygon = shape(geometry)
    centroid = shapely_polygon.centroid
    return centroid.x, centroid.y

def construct_url(county_name, clon, clat, years):
    """Construct the URL for data download based on county properties."""
    base_url = "https://lidar.digitalforestry.org/download.php?"
    if county_name.lower() == 'marion':
        base_url = "https://lidar.digitalforestry.org/download_marion.php?"
    return f"{base_url}cname={county_name.replace(' ', '').lower()}&clon={clon}&clat={clat}&years={years}"

In [ ]:
# optional - to print out links
# def generate_download_links(geojson_path):
#     geojson_data = parse_geojson(geojson_path)
#     links = []
#     for feature in geojson_data['features']:
#         properties = feature['properties']
#         geometry = feature['geometry']
#         county_name = properties['NAME_L']
#         years = properties['lidar_year']
#         clon, clat = calculate_center(geometry)
#         link = construct_url(county_name, clon, clat, years)
#         links.append((county_name, link))
#     return links

# geojson_path = 'in_counties_with_lidar_year.geojson'
# download_links = generate_download_links(geojson_path)
# for county, link in download_links:
#     print(county, link)

In [11]:
# to update the links in my current OpenIndexMap GeoJSON
# specific task underdaken 2024-04; not necessarily something we will need to do again in the future

def update_openindexmap_links(lidar_geojson_path, openindexmap_path, output_path):
    lidar_data = parse_geojson(lidar_geojson_path)
    openindexmap_data = parse_geojson(openindexmap_path)

    # Creating a map of county names to lidar_years from the lidar data
    lidar_map = {feature['properties']['NAME_L']: feature['properties']['lidar_year'] for feature in lidar_data['features']}
    
    for feature in openindexmap_data['features']:
        properties = feature['properties']
        geometry = feature['geometry']
        county_name = properties['title']  # Assuming the county name is stored under 'title'
        
        if county_name in lidar_map:
            clon, clat = calculate_center(geometry)
            years = lidar_map[county_name]
            new_url = construct_url(county_name, clon, clat, years)
            properties['downloadUrl'] = new_url  # Updating the URL in the properties

    # Write the updated GeoJSON back to a file
    with open(output_path, 'w') as file:
        json.dump(openindexmap_data, file, indent=4)

# Example usage
lidar_geojson_path = 'in_counties_with_lidar_year.geojson'
openindexmap_path = '09a-03.geojson'
output_path = 'updated_09a-03.geojson'
update_openindexmap_links(lidar_geojson_path, openindexmap_path, output_path)
print(f"Updated OpenIndexMap GeoJSON saved to {output_path}")


Updated OpenIndexMap GeoJSON saved to updated_09a-03.geojson
